<a href="https://colab.research.google.com/github/NonRoute/Data-Sci-Eng-Project/blob/main/Datasci_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Engineering (DE) Spark

## Spark Preparation

In [70]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [71]:
if IN_COLAB:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    !wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
    !tar xf spark-3.3.2-bin-hadoop3.tgz
    !mv spark-3.3.2-bin-hadoop3 spark
    !pip install -q findspark
    import os
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark"

## Start a Local Cluster

In [72]:
import findspark
findspark.init()

In [73]:
spark_url = 'local'

In [74]:
from pyspark.sql import SparkSession

In [75]:
spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark ML')\
        .getOrCreate()

## Spark SQL Data Preparation

In [76]:
from pyspark import SparkFiles

# Data from https://www.traffy.in.th/?page_id=27351
# Update every 3hr

url = 'https://publicapi.traffy.in.th/dump-csv-chadchart/bangkok_traffy.csv'
spark.sparkContext.addFile(url)

In [77]:
# if url not working
# path = 'bangkok_traffy.csv'

# df = spark.read.option("delimiter", ",").option("multiline", "true").option("quote", '"').option("header", "true").option("escape", "\\").option("escape", '"').csv(path)
# df.show()

In [78]:
df = spark.read.option("delimiter", ",").option("multiline", "true").option("quote", '"').option("header", "true").option("escape", "\\").option("escape", '"').csv("file://" + SparkFiles.get("bangkok_traffy.csv"))
df.show()

+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+----------------+-----------------+--------------------+--------------------+--------------+----+------------+--------------------+
|  ticket_id|                type|        organization|             comment|               photo|         photo_after|            coords|             address|     subdistrict|         district|            province|           timestamp|         state|star|count_reopen|       last_activity|
+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+----------------+-----------------+--------------------+--------------------+--------------+----+------------+--------------------+
|2021-9LHDM6|                  {}|                null|            ไม่มีภาพ|https://storage.g...|                null|100.48661,13

In [79]:
df.count()

255152

In [80]:
df.printSchema()

root
 |-- ticket_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- organization: string (nullable = true)
 |-- comment: string (nullable = true)
 |-- photo: string (nullable = true)
 |-- photo_after: string (nullable = true)
 |-- coords: string (nullable = true)
 |-- address: string (nullable = true)
 |-- subdistrict: string (nullable = true)
 |-- district: string (nullable = true)
 |-- province: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- state: string (nullable = true)
 |-- star: string (nullable = true)
 |-- count_reopen: string (nullable = true)
 |-- last_activity: string (nullable = true)



In [81]:
# drop unused column
cols = ['ticket_id','photo', 'photo_after']
df = df.drop(*cols)

 ## Convert to proper data type

In [82]:
from pyspark.sql.functions import col
cols = ['star', 'count_reopen']
for c in cols:
    df = df.withColumn(c, col(c).cast('int'))

In [83]:
cols = ['timestamp', 'last_activity']
for c in cols:
    df = df.withColumn(c, col(c).cast('timestamp'))

In [84]:
from pyspark.sql.functions import split, regexp_replace
cols = ['type']
for c in cols:
    df = df.withColumn(c, split(regexp_replace(col(c), "[{}]", ""), ","))

In [85]:
cols = ['organization', 'coords']
for c in cols:
  df = df.withColumn(c, split(col(c), ","))

In [86]:
df.printSchema()

root
 |-- type: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- organization: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- comment: string (nullable = true)
 |-- coords: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- address: string (nullable = true)
 |-- subdistrict: string (nullable = true)
 |-- district: string (nullable = true)
 |-- province: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- state: string (nullable = true)
 |-- star: integer (nullable = true)
 |-- count_reopen: integer (nullable = true)
 |-- last_activity: timestamp (nullable = true)



In [87]:
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-----------------+--------------------+--------------------+--------------+----+------------+--------------------+
|                type|        organization|             comment|              coords|             address|     subdistrict|         district|            province|           timestamp|         state|star|count_reopen|       last_activity|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-----------------+--------------------+--------------------+--------------+----+------------+--------------------+
|                  []|                null|            ไม่มีภาพ|[100.48661, 13.79...|1867 จรัญสนิทวงศ์...|         บางพลัด|          บางพลัด|       กรุงเทพมหานคร|2021-09-01 10:44:...|กำลังดำเนินการ|null|        null|2022-02-22 04:59:...|
|         [ความสะอาด]|        [เขตบางซื่อ]|     

## Drop null

In [88]:
# count null
import pyspark.sql.functions as F

df_agg = df.agg(*[F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns])

In [89]:
df_agg.show()

+----+------------+-------+------+-------+-----------+--------+--------+---------+-----+------+------------+-------------+
|type|organization|comment|coords|address|subdistrict|district|province|timestamp|state|  star|count_reopen|last_activity|
+----+------------+-------+------+-------+-----------+--------+--------+---------+-----+------+------------+-------------+
|  97|         969|   2383|     0|   2383|         70|      72|      23|        0|    0|160758|      117732|           14|
+----+------------+-------+------+-------+-----------+--------+--------+---------+-----+------+------------+-------------+



In [90]:
df.filter("last_activity is NULL").show()

+-------------+------------+--------------------+--------------------+--------------------+------------+-----------+--------------------+--------------------+-----------+----+------------+-------------+
|         type|organization|             comment|              coords|             address| subdistrict|   district|            province|           timestamp|      state|star|count_reopen|last_activity|
+-------------+------------+--------------------+--------------------+--------------------+------------+-----------+--------------------+--------------------+-----------+----+------------+-------------+
|        [ถนน]|        null|จอดรถกันข้างทางใน...|[100.50468, 13.70...|48/1 ถ. เจริญกรุง...| วัดพระยาไกร|  บางคอแหลม|       กรุงเทพมหานคร|2022-07-30 07:15:...|รอรับเรื่อง|null|        null|         null|
|           []|        null|     วางของบนทางเท้า|[100.53675, 13.70...|395 4-5 ซอย นราธิ...|   ช่องนนทรี|    ยานนาวา|จังหวัดกรุงเทพมหานคร|2022-08-05 05:53:...|รอรับเรื่อง|null|        null|

In [91]:
# drop rows where last_activity = null
df = df.na.drop(subset=["last_activity"])

In [92]:
df.count()

255138

## calculate time_to_state

In [93]:
from pyspark.sql.functions import expr

# calculate time_to_state
df = df.withColumn("time_to_state", expr("last_activity - timestamp"))
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-----------------+--------------------+--------------------+--------------+----+------------+--------------------+--------------------+
|                type|        organization|             comment|              coords|             address|     subdistrict|         district|            province|           timestamp|         state|star|count_reopen|       last_activity|       time_to_state|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------------+-----------------+--------------------+--------------------+--------------+----+------------+--------------------+--------------------+
|                  []|                null|            ไม่มีภาพ|[100.48661, 13.79...|1867 จรัญสนิทวงศ์...|         บางพลัด|          บางพลัด|       กรุงเทพมหานคร|2021-09-01 10:44:...|กำลังดำเนินการ|null|        null|2022-02

## (Run everything before this, after this is optional)

## Visualization Start here

## Focus on state

In [94]:
df.groupby('state').count().show()

+--------------+------+
|         state| count|
+--------------+------+
|   รอรับเรื่อง| 17655|
|กำลังดำเนินการ| 46163|
|     เสร็จสิ้น|191320|
+--------------+------+



In [95]:
from pyspark.sql.functions import min, avg, max

df.groupby('state').agg(min('time_to_state').alias('min_time_to_state'),
                         avg('time_to_state').alias('avg_time_to_state'),
                         max('time_to_state').alias('max_time_to_state')).show(20, False)


+--------------+-------------------------------------------+-------------------------------------------+--------------------------------------------+
|state         |min_time_to_state                          |avg_time_to_state                          |max_time_to_state                           |
+--------------+-------------------------------------------+-------------------------------------------+--------------------------------------------+
|รอรับเรื่อง   |INTERVAL '-0 00:00:00.030728' DAY TO SECOND|INTERVAL '11 13:03:49.92646' DAY TO SECOND |INTERVAL '313 16:03:34.072568' DAY TO SECOND|
|กำลังดำเนินการ|INTERVAL '0 00:00:30.703422' DAY TO SECOND |INTERVAL '87 18:14:49.015143' DAY TO SECOND|INTERVAL '480 22:43:41.851235' DAY TO SECOND|
|เสร็จสิ้น     |INTERVAL '0 00:00:14.963721' DAY TO SECOND |INTERVAL '43 22:45:42.245733' DAY TO SECOND|INTERVAL '456 06:15:38.086373' DAY TO SECOND|
+--------------+-------------------------------------------+----------------------------------------

In [96]:
from pyspark.sql.functions import year, month

df.groupBy(year('timestamp').alias('year'), month('timestamp').alias('month')).count().orderBy('year', 'month').show()

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|2021|    9|   12|
|2021|   10|    1|
|2021|   11|    7|
|2021|   12|  162|
|2022|    1|  160|
|2022|    2|  172|
|2022|    3|  108|
|2022|    4|   24|
|2022|    5| 2210|
|2022|    6|61633|
|2022|    7|40880|
|2022|    8|27035|
|2022|    9|22323|
|2022|   10|13521|
|2022|   11|11481|
|2022|   12|11380|
|2023|    1|18590|
|2023|    2|18575|
|2023|    3|18213|
|2023|    4| 8651|
+----+-----+-----+



In [97]:
from pyspark.sql.functions import year, month, count

df.groupBy(year('timestamp').alias('year'), month('timestamp').alias('month'), 'state')\
  .agg(count('time_to_state').alias('row_count'),
       min('time_to_state').alias('min_time_to_state'),
       avg('time_to_state').alias('avg_time_to_state'),
       max('time_to_state').alias('max_time_to_state'))\
  .orderBy('state','year', 'month').show(100, False)


+----+-----+--------------+---------+--------------------------------------------+--------------------------------------------+--------------------------------------------+
|year|month|state         |row_count|min_time_to_state                           |avg_time_to_state                           |max_time_to_state                           |
+----+-----+--------------+---------+--------------------------------------------+--------------------------------------------+--------------------------------------------+
|2021|9    |กำลังดำเนินการ|9        |INTERVAL '146 05:28:39.035754' DAY TO SECOND|INTERVAL '155 04:47:42.703305' DAY TO SECOND|INTERVAL '173 18:15:03.269059' DAY TO SECOND|
|2021|11   |กำลังดำเนินการ|7        |INTERVAL '88 19:12:22.087276' DAY TO SECOND |INTERVAL '96 12:49:23.998591' DAY TO SECOND |INTERVAL '106 22:50:33.926335' DAY TO SECOND|
|2021|12   |กำลังดำเนินการ|154      |INTERVAL '53 15:25:27.671055' DAY TO SECOND |INTERVAL '69 15:42:50.610973' DAY TO SECOND |INTERVAL

In [98]:
from pyspark.sql.functions import year, month, count, sum, col

df_count = df.groupBy(year('timestamp').alias('year'), month('timestamp').alias('month'), 'state') \
             .agg(count('time_to_state').alias('row_count'))

df_total_count = df_count.groupBy('year', 'month') \
                         .agg(sum('row_count').alias('total_count'))

df_ratio = df_count.join(df_total_count, ['year', 'month']) \
                   .withColumn('state_ratio', col('row_count') / col('total_count')) \
                   

df_ratio.orderBy('year', 'month','state').show(100, False)

+----+-----+--------------+---------+-----------+--------------------+
|year|month|state         |row_count|total_count|state_ratio         |
+----+-----+--------------+---------+-----------+--------------------+
|2021|9    |กำลังดำเนินการ|9        |12         |0.75                |
|2021|9    |เสร็จสิ้น     |3        |12         |0.25                |
|2021|10   |เสร็จสิ้น     |1        |1          |1.0                 |
|2021|11   |กำลังดำเนินการ|7        |7          |1.0                 |
|2021|12   |กำลังดำเนินการ|154      |162        |0.9506172839506173  |
|2021|12   |เสร็จสิ้น     |8        |162        |0.04938271604938271 |
|2022|1    |กำลังดำเนินการ|142      |160        |0.8875              |
|2022|1    |เสร็จสิ้น     |18       |160        |0.1125              |
|2022|2    |กำลังดำเนินการ|135      |172        |0.7848837209302325  |
|2022|2    |รอรับเรื่อง   |8        |172        |0.046511627906976744|
|2022|2    |เสร็จสิ้น     |29       |172        |0.1686046511627907  |
|2022|

In [99]:
df_ratio.orderBy('state','year', 'month').show(100, False)

+----+-----+--------------+---------+-----------+--------------------+
|year|month|state         |row_count|total_count|state_ratio         |
+----+-----+--------------+---------+-----------+--------------------+
|2021|9    |กำลังดำเนินการ|9        |12         |0.75                |
|2021|11   |กำลังดำเนินการ|7        |7          |1.0                 |
|2021|12   |กำลังดำเนินการ|154      |162        |0.9506172839506173  |
|2022|1    |กำลังดำเนินการ|142      |160        |0.8875              |
|2022|2    |กำลังดำเนินการ|135      |172        |0.7848837209302325  |
|2022|3    |กำลังดำเนินการ|56       |108        |0.5185185185185185  |
|2022|4    |กำลังดำเนินการ|3        |24         |0.125               |
|2022|5    |กำลังดำเนินการ|307      |2210       |0.13891402714932127 |
|2022|6    |กำลังดำเนินการ|10420    |61633      |0.16906527347362615 |
|2022|7    |กำลังดำเนินการ|6472     |40880      |0.15831702544031312 |
|2022|8    |กำลังดำเนินการ|3793     |27035      |0.1402996116145737  |
|2022|

## Focus on type

In [100]:
from pyspark.sql.functions import explode, size

df_exploded = df.select(explode('type').alias('type_exploded'), 'state', 'time_to_state')

df_grouped = df_exploded.groupBy('type_exploded', 'state') \
                        .agg(count('time_to_state').alias('row_count'),
                             min('time_to_state').alias('min_time_to_state'),
                             avg('time_to_state').alias('avg_time_to_state'),
                             max('time_to_state').alias('max_time_to_state')) \
                        .orderBy('type_exploded')

df_grouped.show(100, False)

+-------------+--------------+---------+-------------------------------------------+--------------------------------------------+--------------------------------------------+
|type_exploded|state         |row_count|min_time_to_state                          |avg_time_to_state                           |max_time_to_state                           |
+-------------+--------------+---------+-------------------------------------------+--------------------------------------------+--------------------------------------------+
|             |รอรับเรื่อง   |4427     |INTERVAL '-0 00:00:00.019474' DAY TO SECOND|INTERVAL '11 18:57:09.900526' DAY TO SECOND |INTERVAL '307 20:23:29.802538' DAY TO SECOND|
|             |กำลังดำเนินการ|8470     |INTERVAL '0 00:01:23.341257' DAY TO SECOND |INTERVAL '108 02:43:24.988693' DAY TO SECOND|INTERVAL '480 22:43:41.851235' DAY TO SECOND|
|             |เสร็จสิ้น     |48079    |INTERVAL '0 00:00:14.963721' DAY TO SECOND |INTERVAL '47 00:35:28.708814' DAY TO SECO

## Focus on organization

In [101]:
from pyspark.sql.functions import explode, size

df_exploded = df.select(explode('organization').alias('organization_exploded'), 'state', 'time_to_state')

df_grouped = df_exploded.groupBy('organization_exploded', 'state') \
                        .agg(count('time_to_state').alias('row_count'),
                             min('time_to_state').alias('min_time_to_state'),
                             avg('time_to_state').alias('avg_time_to_state'),
                             max('time_to_state').alias('max_time_to_state')) \
                        .filter(col('state') == 'เสร็จสิ้น') \
                        .orderBy('row_count', ascending=False)

df_grouped.show(100, False)

+--------------------------------------------------------------+---------+---------+------------------------------------------+--------------------------------------------+--------------------------------------------+
|organization_exploded                                         |state    |row_count|min_time_to_state                         |avg_time_to_state                           |max_time_to_state                           |
+--------------------------------------------------------------+---------+---------+------------------------------------------+--------------------------------------------+--------------------------------------------+
|กลุ่มกรุงเทพใต้ (รองปลัดฯ ชาตรี)                              |เสร็จสิ้น|32429    |INTERVAL '0 00:00:22.509931' DAY TO SECOND|INTERVAL '67 19:00:13.261912' DAY TO SECOND |INTERVAL '311 19:41:46.936831' DAY TO SECOND|
|กลุ่มกรุงเทพตะวันออก (รองปลัดฯ ณรงค์)                         |เสร็จสิ้น|31954    |INTERVAL '0 00:00:30.017043' DAY TO SECOND|I

## Focus on district

In [102]:
df_grouped = df.groupBy('district', 'state') \
                        .agg(count('time_to_state').alias('row_count'),
                             min('time_to_state').alias('min_time_to_state'),
                             avg('time_to_state').alias('avg_time_to_state'),
                             max('time_to_state').alias('max_time_to_state')) \
                        .orderBy('district')

df_grouped.show(100, False)

+-----------+--------------+---------+--------------------------------------------+--------------------------------------------+--------------------------------------------+
|district   |state         |row_count|min_time_to_state                           |avg_time_to_state                           |max_time_to_state                           |
+-----------+--------------+---------+--------------------------------------------+--------------------------------------------+--------------------------------------------+
|null       |กำลังดำเนินการ|9        |INTERVAL '39 11:35:20.576196' DAY TO SECOND |INTERVAL '111 06:13:12.151308' DAY TO SECOND|INTERVAL '274 15:52:58.551807' DAY TO SECOND|
|null       |รอรับเรื่อง   |16       |INTERVAL '0 00:00:00.376737' DAY TO SECOND  |INTERVAL '0 04:43:56.520277' DAY TO SECOND  |INTERVAL '2 01:28:28.354655' DAY TO SECOND  |
|null       |เสร็จสิ้น     |47       |INTERVAL '0 00:16:04.067019' DAY TO SECOND  |INTERVAL '67 06:58:15.141717' DAY TO SECOND |IN

In [103]:
# rank by avg_time_to_state

df_grouped = df.groupBy('district', 'state') \
                        .agg(count('time_to_state').alias('row_count'),
                             min('time_to_state').alias('min_time_to_state'),
                             avg('time_to_state').alias('avg_time_to_state'),
                             max('time_to_state').alias('max_time_to_state')) \
                        .filter((col('state') == 'เสร็จสิ้น') & (col('row_count') > 50)) \
                        .orderBy('avg_time_to_state')

df_grouped.show(100, False)

+-----------------+---------+---------+------------------------------------------+--------------------------------------------+--------------------------------------------+
|district         |state    |row_count|min_time_to_state                         |avg_time_to_state                           |max_time_to_state                           |
+-----------------+---------+---------+------------------------------------------+--------------------------------------------+--------------------------------------------+
|ราษฎร์บูรณะ      |เสร็จสิ้น|1955     |INTERVAL '0 00:01:02.493189' DAY TO SECOND|INTERVAL '14 18:08:54.334059' DAY TO SECOND |INTERVAL '291 07:38:27.161993' DAY TO SECOND|
|บางคอแหลม        |เสร็จสิ้น|2335     |INTERVAL '0 00:01:36.057744' DAY TO SECOND|INTERVAL '16 08:03:57.259198' DAY TO SECOND |INTERVAL '290 14:05:44.24771' DAY TO SECOND |
|บางแค            |เสร็จสิ้น|7359     |INTERVAL '0 00:02:14.913236' DAY TO SECOND|INTERVAL '18 03:45:37.632063' DAY TO SECOND |INTERVAL

## Focus on star

In [104]:
df_grouped = df.groupBy('star', 'state') \
                        .agg(count('time_to_state').alias('row_count'),
                             min('time_to_state').alias('min_time_to_state'),
                             avg('time_to_state').alias('avg_time_to_state'),
                             max('time_to_state').alias('max_time_to_state')) \
                        .filter((col('state') == "เสร็จสิ้น")) \
                        .orderBy('star')

df_grouped.show(100, False)

+----+---------+---------+------------------------------------------+-------------------------------------------+--------------------------------------------+
|star|state    |row_count|min_time_to_state                         |avg_time_to_state                          |max_time_to_state                           |
+----+---------+---------+------------------------------------------+-------------------------------------------+--------------------------------------------+
|null|เสร็จสิ้น|98212    |INTERVAL '0 00:00:21.569294' DAY TO SECOND|INTERVAL '46 02:39:48.017073' DAY TO SECOND|INTERVAL '456 06:15:38.086373' DAY TO SECOND|
|1   |เสร็จสิ้น|13169    |INTERVAL '0 00:00:30.758731' DAY TO SECOND|INTERVAL '50 23:38:59.318826' DAY TO SECOND|INTERVAL '373 15:37:09.535253' DAY TO SECOND|
|2   |เสร็จสิ้น|4483     |INTERVAL '0 00:01:06.009175' DAY TO SECOND|INTERVAL '51 07:08:38.927461' DAY TO SECOND|INTERVAL '309 21:49:40.280798' DAY TO SECOND|
|3   |เสร็จสิ้น|9945     |INTERVAL '0 00:00:34

## AI/ML Start predict time to เสร็จสิ้น here

In [105]:
df_done = df.filter(df.state == "เสร็จสิ้น")
df_done.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------+-------------+--------------------+---------+----+------------+--------------------+--------------------+
|                type|        organization|             comment|              coords|             address|subdistrict|district|     province|           timestamp|    state|star|count_reopen|       last_activity|       time_to_state|
+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------+-------------+--------------------+---------+----+------------+--------------------+--------------------+
|         [ความสะอาด]|        [เขตบางซื่อ]|             ขยะเยอะ|[100.53084, 13.81...|12/14 ถนน กรุงเทพ...|       null|    null|กรุงเทพมหานคร|2021-09-03 12:51:...|เสร็จสิ้น|null|        null|2022-06-04 15:34:...|INTERVAL '274 02:...|
|[น้ำท่วม, ร้องเรียน]|[เขตประเวศ, ฝ่ายโ...|น้ำท่วมเวลาฝนตกแล...|[100

In [106]:
df_done.count()

191320